In [1]:
import os
import glob
from osgeo import osr,gdal
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# !pip install ipython-autotime
# %load_ext autotime

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
photon_df = pd.read_csv("/content/drive/My Drive/PS/PS Data/Chandigarh/final.csv", index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
ds = gdal.Open("/content/drive/My Drive/PS/PS Data/Chandigarh/ChandigarhHP_SRTMGL1Ellip.tif")
transform = ds.GetGeoTransform()
band = ds.GetRasterBand(1)
array = band.ReadAsArray()
 
#plt.imshow(array)
array

array([[ 295.16174,  295.1637 ,  295.16562, ..., 1498.1888 , 1495.1909 ,
        1491.1931 ],
       [ 295.16092,  294.16284,  295.1648 , ..., 1481.1864 , 1479.1885 ,
        1473.1907 ],
       [ 293.1601 ,  293.16202,  294.16394, ..., 1467.1838 , 1462.186  ,
        1452.1881 ],
       ...,
       [ 229.7569 ,  228.75735,  228.75784, ...,  644.26385,  638.2654 ,
         626.2669 ],
       [ 228.75552,  228.756  ,  228.75647, ...,  641.2606 ,  633.26215,
         623.2637 ],
       [ 228.75417,  228.75464,  229.75511, ...,  635.25745,  628.2589 ,
         622.26044]], dtype=float32)

In [5]:
width = ds.RasterXSize
height = ds.RasterYSize
print(width)
print(height)

2551
1975


In [6]:
gt = ds.GetGeoTransform()

In [7]:
origin_x = gt[0]
origin_y = gt[3]
pixel_res_x = gt[1]
pixel_res_y = gt[5]

In [8]:
def latitude(n):
    return origin_y + (n/width * pixel_res_y)
def longitude(n):
    return origin_x + (n%width * pixel_res_x)

In [9]:
array = np.array(array)
array = array.flatten()
dem_df = pd.DataFrame(data = array,columns = ['dem height'])
dem_df['index1']=dem_df.index
dem_df['latitude'] = dem_df.index1.apply(latitude)
dem_df['longitude'] = dem_df.index1.apply(longitude)
dem_df = dem_df.drop(['index1'],axis=1)
dem_df = dem_df[['latitude','longitude','dem height']]

In [10]:
dem_df

,latitude,longitude,dem height
0,31.077361,76.668750,295.161743
1,31.077361,76.669028,295.163696
2,31.077361,76.669306,295.165619
3,31.077361,76.669583,295.167542
4,31.077361,76.669861,295.169464
...,...,...,...
5038220,30.528751,77.375972,645.254395
5038221,30.528750,77.376250,642.255920
5038222,30.528750,77.376528,635.257446
5038223,30.528750,77.376806,628.258911


In [11]:
def get_latitude_index(lat):
  return int((lat-origin_y)/pixel_res_y)

def get_longitude_index(lon):
  return int((lon-origin_x)/pixel_res_x)

def get_high_res_latitude_index(lat):
  return int((lat-origin_y)/(pixel_res_y/3))

def get_high_res_longitude_index(lon):
  return int((lon-origin_x)/(pixel_res_x/3))

In [12]:
photon_df['latitude_index']= photon_df.latitude.apply(get_latitude_index)
photon_df['longitude_index']= photon_df.longitude.apply(get_longitude_index)
photon_df['latitude_index2']= photon_df.latitude.apply(get_high_res_latitude_index)
photon_df['longitude_index2']= photon_df.longitude.apply(get_high_res_longitude_index)

In [13]:
photon_df['container_box_id']=(photon_df['latitude_index'])*(width)+photon_df['longitude_index']

In [14]:
photon_df['container_box_id2']=(photon_df['latitude_index2'])*(width*3)+photon_df['longitude_index2']

In [15]:
photon_df

,latitude,longitude,photon height,latitude_index,longitude_index,latitude_index2,longitude_index2,container_box_id,container_box_id2
index,,,,,,,,,
1,31.046138,77.090977,1241.36800,112,1520,337,4560,287232,2583621
3,31.046138,77.090977,1245.69310,112,1520,337,4560,287232,2583621
5,31.046138,77.090977,1245.35240,112,1520,337,4560,287232,2583621
8,31.046125,77.090975,1245.37960,112,1520,337,4560,287232,2583621
18,31.046087,77.090971,1242.52360,112,1519,337,4559,287231,2583620
...,...,...,...,...,...,...,...,...,...
101868,30.535940,77.032295,273.23206,1949,1308,5847,3926,4973207,44751017
101869,30.535927,77.032294,275.11923,1949,1308,5847,3926,4973207,44751017
101870,30.535914,77.032292,275.10530,1949,1308,5847,3926,4973207,44751017


In [16]:
photon_df=photon_df.groupby(['container_box_id2']).mean()
photon_df

,latitude,longitude,photon height,latitude_index,longitude_index,latitude_index2,longitude_index2,container_box_id
container_box_id2,,,,,,,,
2570909,31.046158,76.622524,238.958543,112,-166,336,-499,285546
2571062,31.046172,76.636645,274.511396,112,-115,336,-346,285597
2571134,31.046162,76.643347,272.551715,112,-91,336,-274,285621
2571220,31.046179,76.651286,278.855602,112,-62,336,-188,285650
2571583,31.046168,76.685040,278.166230,112,58,336,175,285770
...,...,...,...,...,...,...,...,...
44755518,30.535879,76.740458,233.468925,1949,258,5848,774,4972157
44758323,30.535870,77.000165,276.659348,1949,1193,5848,3579,4973092
44758695,30.535877,77.034662,273.493391,1949,1317,5848,3951,4973216


In [17]:
photon_df=photon_df.drop(['latitude_index'],axis=1)
photon_df=photon_df.drop(['longitude_index'],axis=1)
photon_df=photon_df.drop(['latitude_index2'],axis=1)
photon_df=photon_df.drop(['longitude_index2'],axis=1)
photon_df.reset_index(drop=True, inplace=True)

In [18]:
photon_df["container_box_id"] = photon_df["container_box_id"].astype(int)

In [19]:
photon_df

,latitude,longitude,photon height,container_box_id
0,31.046158,76.622524,238.958543,285546
1,31.046172,76.636645,274.511396,285597
2,31.046162,76.643347,272.551715,285621
3,31.046179,76.651286,278.855602,285650
4,31.046168,76.685040,278.166230,285770
...,...,...,...,...
422815,30.535879,76.740458,233.468925,4972157
422816,30.535870,77.000165,276.659348,4973092
422817,30.535877,77.034662,273.493391,4973216
422818,30.535871,77.067050,284.208800,4973332


In [20]:
merged_df = photon_df.merge(dem_df, left_on='container_box_id', right_index = True)

In [21]:
merged_df

,latitude_x,longitude_x,photon height,container_box_id,latitude_y,longitude_y,dem height
0,31.046158,76.622524,238.958543,285546,31.046268,77.331250,2252.570557
17,31.046110,76.622538,239.075707,285546,31.046268,77.331250,2252.570557
73,31.046015,76.622534,238.668119,285546,31.046268,77.331250,2252.570557
74,31.046015,76.622552,239.205448,285546,31.046268,77.331250,2252.570557
1,31.046172,76.636645,274.511396,285597,31.046263,77.345417,1839.678345
...,...,...,...,...,...,...,...
422811,30.535959,77.194925,338.025767,4973793,30.535766,77.194861,328.428101
422819,30.535878,77.194917,337.919400,4973793,30.535766,77.194861,328.428101
422812,30.535937,77.197937,332.920336,4973804,30.535765,77.197917,329.440277
422813,30.535943,77.198190,332.895940,4973804,30.535765,77.197917,329.440277


In [22]:
merged_df.drop(columns = ['container_box_id'], inplace = True)
X = merged_df.drop('photon height',axis=1)
y = merged_df['photon height']

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=101)

In [25]:
import numpy as np
y_train = np.array(y_train)
y_test = np.array(y_test)

In [26]:
y_train = y_train.reshape(len(y_train), 1)
y_test = y_test.reshape(len(y_test), 1)

In [27]:
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
X_train= scaler_x.fit_transform(X_train)
X_test = scaler_x.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

In [28]:
X_train.shape

(295974, 5)

In [29]:
X_test.shape

(126846, 5)

In [30]:
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics

In [31]:
model = LinearRegression()

In [32]:
model.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [33]:
finalarray=np.zeros([height*3*width*3], dtype = int)
df= pd.DataFrame(columns=['zero'],data=finalarray)
df

,zero
0,0
1,0
2,0
3,0
4,0
...,...
45344020,0
45344021,0
45344022,0
45344023,0


In [34]:
def latitude_final(n):
    return origin_y + (n/(width*3) * pixel_res_y/3.0)
def longitude_final(n):
    return origin_x + (n%(width*3) * pixel_res_x/3.0)

In [35]:
df['index1']=df.index
df['latitude']=df.index1.apply(latitude_final)
df['longitude']=df.index1.apply(longitude_final)

In [36]:
df=df.drop(columns=['index1','zero'],axis=1)
df['latitude_index']= df.latitude.apply(get_latitude_index)
df['longitude_index']= df.longitude.apply(get_longitude_index)
df['container_box_id']=(df['latitude_index'])*(width)+df['longitude_index']
df=df.drop(['latitude_index'],axis=1)
df=df.drop(['longitude_index'],axis=1)
df["container_box_id"] = df["container_box_id"].astype(int)

In [37]:
final_merged_df = df.merge(dem_df, left_on='container_box_id', right_index = True)
final_merged_df.drop(columns = ['container_box_id'], inplace = True)

In [38]:
final_merged_df

,latitude_x,longitude_x,latitude_y,longitude_y,dem height
0,31.077361,76.668750,31.077361,76.668750,295.161743
1,31.077361,76.668843,31.077361,76.668750,295.161743
2,31.077361,76.668935,31.077361,76.668750,295.161743
7653,31.077269,76.668750,31.077361,76.668750,295.161743
7654,31.077269,76.668843,31.077361,76.668750,295.161743
...,...,...,...,...,...
45336370,30.528843,77.377176,30.528750,77.377083,622.260437
45336371,30.528843,77.377269,30.528750,77.377083,622.260437
45344022,30.528750,77.377083,30.528750,77.377083,622.260437
45344023,30.528750,77.377176,30.528750,77.377083,622.260437


In [39]:
final_merged_df.sort_index()

,latitude_x,longitude_x,latitude_y,longitude_y,dem height
0,31.077361,76.668750,31.077361,76.668750,295.161743
1,31.077361,76.668843,31.077361,76.668750,295.161743
2,31.077361,76.668935,31.077361,76.668750,295.161743
3,31.077361,76.669028,31.077361,76.669028,295.163696
4,31.077361,76.669120,31.077361,76.669028,295.163696
...,...,...,...,...,...
45344020,30.528750,77.376898,30.528750,77.376806,628.258911
45344021,30.528750,77.376991,30.528750,77.376806,628.258911
45344022,30.528750,77.377083,30.528750,77.377083,622.260437
45344023,30.528750,77.377176,30.528750,77.377083,622.260437


In [40]:
final_merged_df.head(6963)

,latitude_x,longitude_x,latitude_y,longitude_y,dem height
0,31.077361,76.668750,31.077361,76.668750,295.161743
1,31.077361,76.668843,31.077361,76.668750,295.161743
2,31.077361,76.668935,31.077361,76.668750,295.161743
7653,31.077269,76.668750,31.077361,76.668750,295.161743
7654,31.077269,76.668843,31.077361,76.668750,295.161743
...,...,...,...,...,...
17624,31.077148,76.883380,31.077277,76.883194,1029.920898
17625,31.077148,76.883472,31.077277,76.883194,1029.920898
2320,31.077333,76.883565,31.077277,76.883472,1028.923462
2321,31.077333,76.883657,31.077277,76.883472,1028.923462


In [41]:
X = final_merged_df
X = scaler_x.transform(X)

In [ ]:
predictions = model.predict(X)
final_merged_df['predictions']=predictions
final_merged_df

In [ ]:
final_merged_df['height']=scaler_y.inverse_transform(final_merged_df['predictions'].values.reshape(-1, 1))

In [ ]:
final_merged_df

In [ ]:
final_merged_df=final_merged_df.sort_index()

In [ ]:
from google.colab import files

# df.to_csv('final_merged_df.csv')
# files.download('final_merged_df.csv')

In [ ]:
final_merged_df=final_merged_df.drop(columns=['latitude_x','longitude_x','latitude_y','longitude_y','predictions','dem height'])

In [ ]:
final_merged_df

In [ ]:
finalarray = final_merged_df.to_numpy()
finalarray=finalarray.reshape(height*3 ,width*3)
finalarray

In [ ]:
dst_filename = 'Chandigarh2linearregression.tiff'
x_pixels = width*3  # number of pixels in x
y_pixels = height*3  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
dataset.GetRasterBand(1).WriteArray(finalarray)

In [ ]:
gt

In [ ]:
gtlist = list(gt)
gtlist[1]=gtlist[1]/3
gtlist[5]=gtlist[5]/3
gt=tuple(gtlist)

In [ ]:
geotrans = gt  #get GeoTranform from existed 'data0'
proj=ds.GetProjection() #you can get from a exsited tif or import 
dataset.SetGeoTransform(geotrans)
dataset.SetProjection(proj)
dataset.FlushCache()
dataset=None

In [ ]:
ls

In [ ]:
from google.colab import files
files.download('Chandigarh2linearregression.tiff')